In [21]:
class OrnsteinUhlenbeckProcess():
    def __init__(self, theta, mu=0., sigma=1., dt=1e-2, x0=None, size=1, sigma_min=0.2, n_steps_annealing=1000):
        self.theta = theta
        self.mu = mu
        self.dt = dt
        self.x0 = x0
        self.size = size
        self.n_steps = 0

        if sigma_min is not None:
            self.m = -float(sigma - sigma_min) / float(n_steps_annealing)
            self.c = sigma
            self.sigma_min = sigma_min
        else:
            self.m = 0.
            self.c = sigma
            self.sigma_min = sigma

        self.reset_states()

    def sample(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + self.current_sigma * np.sqrt(self.dt) * np.random.normal(size=self.size)
        self.x_prev = x
        self.n_steps += 1
        return x

    def reset_states(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros(self.size)
    @property
    def current_sigma(self):
        sigma = max(self.sigma_min, self.m * float(self.n_steps) + self.c)
        return sigma


In [22]:
import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple
from tqdm import tqdm_notebook as tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical, Normal
import warnings
warnings.filterwarnings("ignore")

# Cart Pole
'''
parser = argparse.ArgumentParser(description='PyTorch actor-critic example')
parser.add_argument('--gamma', type=float, default=0.99, metavar='G',
                    help='discount factor (default: 0.99)')
parser.add_argument('--seed', type=int, default=543, metavar='N',
                    help='random seed (default: 543)')
parser.add_argument('--render', action='store_true',
                    help='render the environment')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='interval between training status logs (default: 10)')
args = parser.parse_args()
'''

env = gym.make('LunarLanderContinuous-v2')
env.seed(1)
torch.manual_seed(1)


SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])


class Policy(nn.Module):
    """
    implements both actor and critic in one model
    """
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(8, 128)

        # actor's layer
        self.action_head = nn.Linear(128, 2)

        # critic's layer
        self.value_head = nn.Linear(128, 1)

        # action & reward buffer
        self.saved_actions = []
        self.rewards = []
        
        self.noise_gen = OrnsteinUhlenbeckProcess(theta = 0.25 , sigma_min=1.0)
        
    def forward(self, x):
        """
        forward of both actor and critic
        """
        x = F.relu(self.affine1(x))

        # actor: choses action to take from state s_t 
        # by returning probability of each action
        action_mean = self.action_head(x)[0]

        # critic: evaluates being in the state s_t
        state_values = self.value_head(x)

        # return values for both actor and critic as a tupel of 2 values:
        # 1. a list with the probability of each action over the action space
        # 2. the value from state s_t 
        return action_mean, state_values


model = Policy()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()


def select_action(state , exploration = True):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)

    # create a categorical distribution over the list of probabilities of actions
    if exploration:
        probs = probs + torch.from_numpy(model.noise_gen.sample())
        
    m = Normal(probs , torch.tensor([0.2, 0.2]))

    # and sample an action using the distribution
    action = m.sample()
        
    # save to action buffer
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))

    # the action to take (left or right)
    return action.numpy()


def finish_episode():
    """
    Training code. Calcultes actor and critic loss and performs backprop.
    """
    R = 0
    saved_actions = model.saved_actions
    policy_losses = [] # list to save actor (policy) loss
    value_losses = [] # list to save critic (value) loss
    returns = [] # list to save the true values

    # calculate the true value using rewards returned from the environment
    for r in model.rewards[::-1]:
        # calculate the discounted value
        R = r + 0.99 * R
        returns.insert(0, R)

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()

        # calculate actor (policy) loss 
        policy_losses.append(-log_prob * advantage)

        # calculate critic (value) loss using L1 smooth loss
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))

    # reset gradients
    optimizer.zero_grad()

    # sum up all the values of policy_losses and value_losses
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()

    # perform backprop
    loss.backward()
    optimizer.step()

    # reset rewards and action buffer
    del model.rewards[:]
    del model.saved_actions[:]

try:
    running_reward = 10

    # run inifinitely many episodes
    for i_episode in tqdm(range(500)):

        # reset environment and episode reward
        state = env.reset()
        ep_reward = 0

        # for each episode, only run 9999 steps so that we don't 
        # infinite loop while learning
        for t in range(1, 1000):

            # select action from policy
            action = select_action(state , False)
            # take the action
            state, reward, done, _ = env.step(action)
            if i_episode % 10 == 0:
                env.render()
                
            acc_reward += reward
            shaped_reward = float(reward) - abs(state[1])*2 - abs(state[0])
            if (state[1]<1.8):
                shaped_reward = float(reward) - abs(state[3])
                
            if (done):
                shaped_reward = float(reward)+200
                
            if (state[6]==1 or state[7]==1):
                shaped_reward = float(reward)+10
            
            model.rewards.append(shaped_reward)
            ep_reward += reward
            if done:
                break

        # update cumulative reward
        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward

        # perform backprop
        finish_episode()

        # log results
        if i_episode % 10 == 0:
            print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward))

        # check if we have "solved" the cart pole problem
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break
finally:
    env.close()



Episode 0	Last reward: -134.12	Average reward: 2.79
Episode 10	Last reward: -783.54	Average reward: -466.29
Episode 20	Last reward: -1691.98	Average reward: -791.57
Episode 30	Last reward: -894.24	Average reward: -906.74
Episode 40	Last reward: -2166.11	Average reward: -978.24
Episode 50	Last reward: -1513.93	Average reward: -1068.97
Episode 60	Last reward: -1089.08	Average reward: -1018.14


KeyboardInterrupt: 

In [10]:
try:
    for i_episode in tqdm(range(500)):

        # reset environment and episode reward
        state = env.reset()

        for t in range(1, 1000):

            # select action from policy
            action = select_action(state , False)
            # take the action
            state, reward, done, _ = env.step([action])
            if i_episode % 10 == 0:
                env.render()
            if done:
                break

finally:
    env.close()